# Final Project for Cogs 118B
Group Members:
- Samruddhi Hande ([email](shande@ucsd.edu))
- Ron Hasson ([email](rhasson@ucsd.edu))
- Andrew Hernandez ([email](ash053@ucsd.edu))
- Mehail Mathew Sunny ([email](msmathew@ucsd.edu))
- Justin Yang ([email](justin-yang@ucsd.edu))

## Agglomerative clustering algorithm
### Import packages and set up paths

In [ ]:
import os, re
import pandas as pd
import numpy as np
import time
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import fowlkes_mallows_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
raw_npca = np.load('data/sketches_raw_nopca.npy')
fc6_npca = np.load('data/sketches_fc6_nopca.npy')
raw_pca  = np.load('data/sketches_raw_pca.npy')
fc6_pca  = np.load('data/sketches_fc6_pca.npy')
metadata = pd.read_csv('data/sketches_metadata.csv')
metadata['category_factored'] = LabelEncoder().fit_transform(metadata.category)

feature_set = [raw_npca, fc6_npca, raw_pca, fc6_pca]
labels_set  = [metadata.sort_values(col).category_factored.values 
               for col in metadata.drop(columns=['category', 'category_factored']).columns]

In [ ]:
model = AgglomerativeClustering(n_clusters = 32, affinity = 'euclidean', linkage = 'ward').fit(feature_set[3])
print(f'ward, euclidean: ', fowlkes_mallows_score(labels_set[3], model.labels_))

for i in ["euclidean", "manhattan", "cosine"]:
    for j in ['average', 'complete', 'single']:
        model = AgglomerativeClustering(n_clusters = 32, affinity = i, linkage = j).fit(feature_set[3])
        print(f'{j}, {i}: ', fowlkes_mallows_score(labels_set[3], model.labels_))    

#### We will go with agglomerative clustering using cosine linkage metric and 'single' linkage criterion.

In [ ]:
cols = metadata.drop(columns=['category', 'category_factored']).columns.values
FMI = dict()

for i, feature in enumerate(cols):
    model = AgglomerativeClustering(n_clusters = 32, affinity = 'cosine', linkage = 'single').fit(feature_set[i])
    FMI[feature[:-4]] = fowlkes_mallows_score(labels_set[i], model.labels_)
    print(f'On feature {feature[:-4]}, score: {FMI[feature[:-4]]}')

In [ ]:
fowlkes_mallows_score(labels_set['raw_sketch'], model.labels_)